In [15]:
import numpy as np
import math
from scipy.stats import norm, gamma, poisson
import time

In [22]:
def PCOUP_SMC(Xdata, epss, k, run, OX):
    """ Code for running SMC-ABC for partially coupled household epidemics"""
    #initialize empty zero array
    OUTPUT = np.zeros(((2*epss[1]+1)*run, 6), dtype = float)
    simcount=0
    count=0
    jj=0
    #Setting standard deviation for importance sampling
    VL = np.sum(OX[:,2]**2*OX[:,3])-np.sum(OX[:,2]*OX[:,3])**2
    sL = math.sqrt(2*VL)
    # Number of samples stored from run acceptances.
    cox = OX[:, 0].size
    
    while jj<run:
        simcount+=1
        LA = np.random.choice(cox, 1, p = OX[:,3]/OX[:,3].sum(), replace = True)[0]
        lambda_L = np.random.normal(OX[LA-1, 2], sL, 1)[0]
        if lambda_L>0:
            J=House_COUP(Xdata,epss[1],lambda_L,k) # Run coupled simulations
            W = J[J[:,3]<J[:, 4],:]# W contains successful simulations (infect close to xA individuals)
            if W.size == 5:
                W = np.array(W).reshape((-1, 5))
            if W[:,0].size>0:
                if  min(W[:, 1])<=epss[0]:
                    jj+=1
                    for ii in range(W[:,0].size):
                        if W[ii, 1]<=epss[0]:
                            count+=1
                            OUTPUT[count-1,:] = np.r_[W[ii, 1:5], lambda_L, np.exp(-lambda_L)/sum(norm.pdf(lambda_L, OX[:,2], sL))]
            
        
    return {'OUTPUT':OUTPUT[0:count,:], 'simcount':simcount}


def PCOUP(Xdata, epss, k, run):
    """Partially coupled ABC algorithm to obtain run accepted values."""
     #initialize empty zero array
    output = np.zeros(((2*epss[1]+1)*run, 5), dtype = float)
    simcount=0
    count=0
    jj=0
    
    while jj<run:
        simcount+=1
        lambda_L = np.random.exponential(1,1) # Sample lambda_L
        J = House_COUP(Xdata,epss[1],lambda_L,k) # Run coupled simulations
        W = J[J[:,3]<J[:,4],:]        # W contains successful simulations (infect close to xA individuals)
        if W.size == 5:
            W = np.array(W).reshape((-1, 5))
        if W[:,0].size > 0:
            if min(W[:, 1])<=epss[0]:
                jj+=1
                for ii in range(W[:, 0].size):
                    if W[ii, 1]<=epss[0]:
                        count+=1
                        output[count-1,:]=np.r_[W[ii,1:5], lambda_L]
                        print(jj,count,simcount)
                        
    # Stores values from simulation - these include closeness of simulated epidemic 
    # to data, range of lambda_G values and lambda_L
    return {'OUTPUT':output[0:count,:], 'simcount':simcount}



def Mexp(k,Lambda,U,a,b):
    """Moment calculator for Exp(\lambda) prior."""
    if k == 0:
        U+=math.exp(-a*Lambda)-math.exp(-Lambda*b)
    if k>0:
        U+=a**k*math.exp(-Lambda*a)-b**k*math.exp(-Lambda*b)+k/Lambda*Mexp((k-1), Lambda, U, a, b)
        
    return U

def House_imp(Xdata, epsil, k, run, OX):
    """Code for running ABC for households with importance sampling"""
    OUTPUT = np.zeros((run, 3))
    hs = np.sum(Xdata, axis=0).astype('int')
    isB = np.sum(Xdata, axis=1).astype('int')
    rowB = np.arange(hs.shape[0] + 1)
    xB = np.sum(np.multiply(isB, rowB))

    simcount, j = 0, 0
    # Compute variances for the pertubations. Bivariate Gaussian
    meanG = np.sum(np.multiply(OX[:, 0], OX[:, 2])) / np.sum(OX[:, 2])
    meanL = np.sum(np.multiply(OX[:, 1], OX[:, 2])) / np.sum(OX[:, 2])
    vG = np.sum(np.multiply(OX[:, 0] ** 2, OX[:, 2])) / np.sum(OX[:, 2]) - meanG ** 2
    vL = np.sum(np.multiply(OX[:, 1] ** 2, OX[:, 2])) / np.sum(OX[:, 2]) - meanL ** 2
    vLG = np.sum(OX[:, 0] * OX[:, 1] * OX[:, 2]) / np.sum(OX[:, 2]) - meanG * meanL
    vaR = 2 * np.array([vG, vLG, vLG, vL]).reshape((2, 2)).T
    Sinv = np.linalg.inv(vaR)
    sG = math.sqrt(2 * vG)
    sLL = math.sqrt((vL - vLG ** 2 / vG) * 2)
    sAL = vLG / vG

    while j < run:
        # print(simcount)
        simcount += 1
        LA = np.random.choice(run, 1, p=OX[:, 2] / OX[:, 2].sum(), replace=True)[0]
        lambda_G = np.random.normal(OX[LA - 1, 0], sG, 1)[0]
        lambda_L = np.random.normal((OX[LA - 1, 1] + sAL * (lambda_G - OX[LA - 1, 0])), sLL, 1)[0]

        if lambda_G > 0 and lambda_L > 0:
            J = House_SEL(hs, lambda_G, lambda_L, k)
            if np.sum(abs(J - Xdata)) <= epsil[0]:
                isJ = np.sum(J, axis=1).astype('int')
                # if (abs(sum(isJ * rowB) - xB) <= epsil[2])
                if abs(sum(isJ * rowB) - xB) <= epsil[1]:
                    j += 1
                    weiZ = 0
                    for i in range(1, run + 1):
                        xDIF = np.array([lambda_G, lambda_L]) - OX[LA - 1, 0:2]
                        mult = xDIF.T @ Sinv @ xDIF
                        weiZ = weiZ + np.exp(-mult / 2)
                    weiG = np.exp(-1 * (lambda_L + lambda_G)) / weiZ
                    OUTPUT[j - 1, :] = [lambda_G, lambda_L, weiG]
                    print(j, simcount)

    return {'OUTPUT': OUTPUT, 'simcount': simcount}

                
def House_COUP(Xdata,epsil,lambda_L,k):
    """Partially coupled ABC algorithm for household epidemics
    lambda_L is drawn from the prior (or however)
    Code finds lambda_G values consistent with the data
    Input: Xdata - Epidemic data to compare simulations with.
    epsil - Max distance between simulated and observed final size for a simulation 
    to be accepted. (Tighter control on distance after simulations straightforward).
    lambda_L - local infection (household) rate
    k - Gamma(k,k) infectious period with k=0 a constant infectious period."""
    hsA = np.sum(Xdata, axis = 0) # hsA[i] - Number of households of size i
    isA = np.sum(Xdata, axis = 1) # isA[i] - Number of households with i-1 infectives
    colA = np.arange(1, hsA.shape[0]+1)
    rowA = np.arange(0, hsA.shape[0]+1)
    
    xA = np.sum(isA*rowA) # Final size
    HH = hsA.shape[0] # HH maximum household size
    ks = np.arange(1, HH+1)
    
    n = np.repeat(ks, hsA)
    m = n.shape[0]# Number of households
    N = np.sum(n) # Population size
    NS = N.copy() # Number of susceptibles
    sev=0       # Running tally of severity (sum of infectious periods)
    threshold=0 # Running tally of (global) threshold required for the next infection
    
    ni = np.repeat(0, n.size) # infectives (per household)
    ns = n.copy() # susceptibles (per household)
    
    OUT = np.zeros((HH+1, HH))
    OUT[0, :] = hsA # Epidemic data in the same form as Xdata
                   # Start with everybody susceptible
    
    DISS = np.zeros((2*epsil+1, 5)) # Matrix for collecting epidemics infecting within epsil of xA infectives.
    SEVI = np.zeros((N, 3)) # Matrix to keep track of number of infectives, severity and threshold.
    ys=0    # number of infectives
    count=0 # number of global infections taking place. First global infection is the introductory case. 
    
    while ys<=(xA+epsil):
        # Only need to consider the epidemic until xA+epsil infections occur.
        # We simulate successive global infections (should they occur) with associated
        # local (within household epidemics)
        # For the count+1 global infection to take place, we require that 
        # for k=1,2,..., count;  lambda_G * severity from first k infectives is larger
        # than the k^th threshold
        count+=1
        kk = np.random.choice(m, 1, p = ns/ns.sum(), replace = True)[0]
        OUT[ni[kk-1],n[kk-1]-1]=OUT[ni[kk-1],n[kk-1]-1]-1
        hou_epi=House_epi(ns[kk-1],k,lambda_L)# Simulate a household epidemic among the remaining susceptibles in the household
        
        ns[kk-1]-=hou_epi[0]
        ni[kk-1] = n[kk-1] - ns[kk-1]#update household kk data (susceptibles and infectives)
        
        OUT[ni[kk-1],n[kk-1]-1]+=1# Update the state of the population following the global 
        #infection and resulting household epidemic
        NS = ns.sum()
        threshold+=np.random.exponential(size = 1, scale = (N/NS))
        
        ys+=hou_epi[0]
        sev+=hou_epi[1]
        if count<=N:
            SEVI[count-1,:] = [ys,sev,threshold]
        # If the number infected is close to xA, we check what value of lambda_G 
        # would be needed for an epidemic of the desired size. 
        # Note that in many cases no value of lambda_G will result in an epidemic 
        # close to xA. 
        if abs(ys-xA)<=epsil:
            dist = np.sum(abs(OUT-Xdata))
            TT = SEVI[0:count, 2]/SEVI[0:count, 1] #ratio of threshold to severity
            Tlow = max(TT[0:(count-1)])
            Thi=TT[0:count].max()   #  Thi is the maximum lambda_G which leads to at most count global infections
            DISS[ys-(xA-epsil), :] = [1,dist,abs(ys-xA),Tlow,Thi]
            
    return DISS


def process(DATA):
    hsA = np.sum(DATA,axis=0)
    isA = np.sum(DATA,axis=1)
    colA = np.array(range(1,len(hsA)+1))
    rowA = np.array(range(0,len(hsA)+1))
    
    xA = np.sum(isA*rowA) # Final size
    N = np.sum(hsA*colA) # Population size
    
    return hsA, isA, colA, rowA, xA, N

def House_epi(n,k,lambda_L):
    
    i=0
    sev=0
    
    if n == 1:
        i = 1
        if k == 0:
            sev = 1
        if k > 0:
            sev = np.random.gamma(k, 1/k, 1)
    
    if n > 1:
        t = thresH(n)
        if k == 0:
            q = np.repeat(1.0,n)
        if k > 0:
            q = np.random.gamma(size=n, shape=k, scale=1/k)
        t = np.r_[t, 2*lambda_L*q.sum()]
        
        i = 0
        test = 0
        while test == 0:
            i+= 1
            if t[i-1] > (lambda_L*np.sum(q[0:i])):
                test = 1 
                sev = np.sum(q[0:i])
                
    return i, sev
# Code for setting (local) thresholds in a household of size n.
# 
# Note local thresholds not required for households of size 1.
# Infection rate does not depend upon households size.
#

def Mexp(k,Lambda,U,a,b):
    """Moment calculator for Exp(\lambda) prior."""
    if k == 0:
        U+=math.exp(-a*Lambda)-math.exp(-Lambda*b)
    if k>0:
        U+=a**k*math.exp(-Lambda*a)-b**k*math.exp(-Lambda*b)+k/Lambda*Mexp((k-1), Lambda, U, a, b)
        
    return U

def thresH(n):
    thres = np.repeat(0.0,n-1)
    thres[0] = np.random.exponential(size=1,scale=1/(n-1))
    if n > 2:
        for i in range(1,n-1):
            thres[i] = thres[i-1] + np.random.exponential(size=1, scale=1/(n-i))
    
    return thres

In [23]:
Tecumseh1 = np.zeros((8,7), dtype = int)
Tecumseh1[0,0:5] = np.array((66 ,87 ,  25  , 22   , 4 ))
Tecumseh1[1,0:5] = np.array((13 ,14 ,  15  ,  9   , 4 ))
Tecumseh1[2,1:6] = np.array((4 ,   4  ,  9  ,  2 ,   1 ))
Tecumseh1[3,2:7] = np.array(( 4   , 3   , 1  ,  1 ,   1))
Tecumseh1[4,3:5] = np.array(( 1,1))


In [42]:
tstartC = time.time()
DATA=Tecumseh1
proc = process(DATA)
hsA, isA , colA, rowA,  xA , N = proc

# Set thresholds
Eps1=[100,70,50]
Eps2=[10,6,4]

#Eps1= [20,12,8]
#Eps2= [3,2,1]

TT=len(Eps1)
EpsM= np.vstack((Eps1, Eps2)).T
# Set infectious period (k) and  number of iterations (run)
k=0
run=500

#
# Set infectious period (k) and
# number of iterations (run)
#

OUTP=PCOUP(DATA,EpsM[0,:],k,run)
OUT = OUTP['OUTPUT']
simT = OUTP['simcount']
OL = np.zeros((OUT[:, 0].size,4))
temp = OUT[:,2:5]
OL[:,0:3] = temp
OL[:,3]=np.exp(-OUT[:,2])-np.exp(-OUT[:,3])
OL[:,3]=OL[:,3]/OL[:,3].sum()

if TT > 1:
    for t in range(1, TT):
        OUTP = PCOUP_SMC(DATA,EpsM[t,:],k,run,OL)
        OUT = OUTP['OUTPUT']
        OL = np.zeros((OUT[:, 0].size, 4))
        OL[:, 0:4] = OUT[:, 2:6]
        OL[:, 3] = (np.exp(-OUT[:,2])-np.exp(-OUT[:,3]))*OL[:,3]
        OL[:, 3] = OL[:, 3]/OL[:, 3].sum()
        simT+=OUTP['simcount']

        
tendC=time.time()
tendC-tstartC

1 1 10
1 2 10
1 3 10
1 4 10
2 5 21
2 6 21
2 7 21
2 8 21
2 9 21
2 10 21
3 11 44
3 12 44
3 13 44
3 14 44
4 15 57
4 16 57
4 17 57
4 18 57
5 19 66
5 20 66
5 21 66
6 22 77
6 23 77
6 24 77
7 25 92
7 26 92
7 27 92
7 28 92
7 29 92
7 30 92
7 31 92
7 32 92
8 33 93
8 34 93
8 35 93
8 36 93
8 37 93
8 38 93
8 39 93
8 40 93
8 41 93
9 42 113
9 43 113
9 44 113
10 45 126
10 46 126
10 47 126
10 48 126
11 49 141
11 50 141
11 51 141
11 52 141
11 53 141
12 54 143
12 55 143
12 56 143
13 57 155
13 58 155
14 59 157
14 60 157
14 61 157
14 62 157
14 63 157
14 64 157
14 65 157
15 66 163
15 67 163
15 68 163
15 69 163
16 70 170
16 71 170
16 72 170
16 73 170
16 74 170
16 75 170
17 76 193
17 77 193
17 78 193
17 79 193
17 80 193
17 81 193
17 82 193
17 83 193
18 84 197
18 85 197
19 86 200
19 87 200
20 88 201
21 89 203
22 90 232
22 91 232
22 92 232
22 93 232
23 94 244
23 95 244
23 96 244
23 97 244
23 98 244
24 99 250
24 100 250
24 101 250
25 102 252
25 103 252
26 104 257
26 105 257
26 106 257
26 107 257
27 108 273
27 10

178 705 2327
179 706 2330
179 707 2330
179 708 2330
180 709 2333
180 710 2333
180 711 2333
181 712 2343
181 713 2343
182 714 2349
182 715 2349
183 716 2356
183 717 2356
183 718 2356
183 719 2356
183 720 2356
184 721 2361
184 722 2361
184 723 2361
185 724 2389
185 725 2389
186 726 2392
186 727 2392
186 728 2392
187 729 2401
187 730 2401
187 731 2401
187 732 2401
187 733 2401
187 734 2401
188 735 2409
188 736 2409
188 737 2409
189 738 2420
189 739 2420
189 740 2420
189 741 2420
190 742 2422
190 743 2422
190 744 2422
190 745 2422
191 746 2444
192 747 2446
192 748 2446
192 749 2446
192 750 2446
192 751 2446
192 752 2446
192 753 2446
193 754 2447
193 755 2447
193 756 2447
194 757 2476
194 758 2476
194 759 2476
194 760 2476
194 761 2476
195 762 2485
195 763 2485
195 764 2485
196 765 2490
196 766 2490
197 767 2494
197 768 2494
198 769 2516
198 770 2516
199 771 2551
199 772 2551
200 773 2553
200 774 2553
201 775 2560
201 776 2560
201 777 2560
201 778 2560
202 779 2567
202 780 2567
202 781 2567

357 1317 4488
357 1318 4488
357 1319 4488
358 1320 4493
359 1321 4495
359 1322 4495
359 1323 4495
359 1324 4495
359 1325 4495
359 1326 4495
359 1327 4495
360 1328 4500
360 1329 4500
360 1330 4500
360 1331 4500
360 1332 4500
360 1333 4500
360 1334 4500
361 1335 4524
361 1336 4524
362 1337 4527
362 1338 4527
362 1339 4527
362 1340 4527
362 1341 4527
362 1342 4527
362 1343 4527
362 1344 4527
363 1345 4564
363 1346 4564
363 1347 4564
363 1348 4564
363 1349 4564
363 1350 4564
363 1351 4564
364 1352 4565
364 1353 4565
364 1354 4565
364 1355 4565
365 1356 4575
365 1357 4575
365 1358 4575
365 1359 4575
365 1360 4575
366 1361 4580
367 1362 4613
367 1363 4613
367 1364 4613
367 1365 4613
367 1366 4613
367 1367 4613
367 1368 4613
367 1369 4613
368 1370 4623
369 1371 4629
370 1372 4632
370 1373 4632
370 1374 4632
370 1375 4632
371 1376 4638
371 1377 4638
371 1378 4638
371 1379 4638
371 1380 4638
371 1381 4638
371 1382 4638
371 1383 4638
371 1384 4638
372 1385 4653
372 1386 4653
372 1387 4653
372 13

1091.0140233039856

In [33]:
# Compute posterior mean and standard deviation
wei = 0
moMG = np.repeat(0.0, 2)
moML = np.repeat(0.0, 2)
Count = OUT[:, 0].size

for i in range(Count):
    wei+=Mexp(0,1,0,OUT[i,2],OUT[i,3])*OUT[i,5]
    for j in range(2):
        moMG[j]=moMG[j]+Mexp(j,1,0,OUT[i,2],OUT[i,3])*OUT[i,5]
        moML[j]=moML[j]+Mexp(0,1,0,OUT[i,2],OUT[i,3])*(OUT[i,4]**(j+1))*OUT[i,5]
        
meanG = moMG[0]/wei
sdG = math.sqrt(moMG[1]/wei-meanG**2)
meanL = moML[0]/wei
sdL = math.sqrt(moML[1]/wei-meanL**2)

# Computes transformed means and standard deviations of
# q_G = exp(-lambdaG * xA/N); q_L = exp(-lambdaL)
#

A1 = 1+xA/N
A2 = 1+2*xA/N

WEIq=(np.exp(-OUT[:,2])-np.exp(-OUT[:,3]))*OUT[:,5]
moqG=[]
moqG.append(sum((np.exp(-A1*OUT[:,2])-np.exp(-A1*OUT[:,3]))*OUT[:,5])/A1)
moqG.append(sum((np.exp(-A2*OUT[:,2])-np.exp(-A2*OUT[:,3]))*OUT[:,5])/A2)

moqL=[]
moqL.append(sum(WEIq*np.exp(-OUT[:,4])))
moqL.append(sum(WEIq*np.exp(-2*OUT[:,4])))

meanqG = moqG[0]/wei
sdqG = math.sqrt(moqG[0]/wei-meanqG**2)
meanqL = moqL[0]/wei
sdqL = math.sqrt(moqL[1]/wei-meanqL**2)

# Summarise results
print("Summarise results", simT)

# Parameter means and sd
print("Parameter means and sd", meanG, sdG, meanL, sdL)

# Transformed parameters means and sd (compare with Clancy and O'Neill (2008)
# and Neal (2012))
print("Transformed parameters means and sd", meanqG, sdqG, meanqL, sdqL)

# Time taken.


0.0010017530589873948

In [34]:
moMG

array([0.00100175, 0.00092164])

In [36]:
tendC=time.time()
(tendC-tstartC)/3

1216.7250076134999

In [37]:
OUT

array([[4.20000000e+01, 4.00000000e+00, 9.66016701e-01, 9.67484319e-01,
        9.15526156e-02, 8.60959730e-05],
       [4.20000000e+01, 2.00000000e+00, 9.67484319e-01, 9.74701974e-01,
        9.15526156e-02, 8.60959730e-05],
       [5.00000000e+01, 3.00000000e+00, 9.01465693e-01, 9.02575403e-01,
        1.50774726e-01, 7.60184272e-05],
       ...,
       [4.40000000e+01, 4.00000000e+00, 8.43230676e-01, 8.56734624e-01,
        2.07604877e-01, 7.75063555e-05],
       [4.40000000e+01, 2.00000000e+00, 8.66703179e-01, 8.74255833e-01,
        1.19621403e-01, 7.96612095e-05],
       [4.60000000e+01, 4.00000000e+00, 7.91540921e-01, 7.93700511e-01,
        1.32983882e-01, 7.76856987e-05]])

In [44]:
np.savetxt('test1.out', OUT, delimiter=',')   # X is an array

In [43]:
OUT.shape

(1093, 6)